# Project 3

In [1]:
import pandas as pd
from typing import List
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gradio as gr
from gradio_calendar import Calendar
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sia = SentimentIntensityAnalyzer()

### Define functions

In [2]:
def calculate_sentiment_score(text: str) -> float:
    """
    Calculate sentiment score for the text

    :param str text: The text to run sentiment analysis on
    :return: The sentiment score
    :rtype: float
    """
    # From VADER documentation:
    #
    # The compound score is computed by summing the valence scores of each word
    # in the lexicon, adjusted according to the rules, and then normalized to be
    # between -1 (most extreme negative) and +1 (most extreme positive). This is
    # the most useful metric if you want a single unidimensional measure of
    # sentiment for a given sentence. Calling it a 'normalized, weighted
    # composite score' is accurate.
    
    sentiment_dict = sia.polarity_scores(text)
    score = sentiment_dict['compound']
    return score

def generate_chart(ticker, start_date, end_date):
    # Loading stock and tweet data
    stock = pd.read_csv('data\stock_data.csv')
    tweets = pd.read_csv('./data/daily-sentiment-summary.csv')
    tweets["Ticker"] = tweets["Ticker"].apply(lambda x: x.replace("APPL", "AAPL"))
    
    # For comparing tz-naive and tz-aware datetime-like objects
    stock['date'] = pd.to_datetime(stock['date']).dt.tz_localize(None)
    tweets['Date'] = pd.to_datetime(tweets['Date']).dt.tz_localize(None)
    start_date = pd.to_datetime(start_date).tz_localize(None)
    end_date = pd.to_datetime(end_date).tz_localize(None)

    # Convert date columns to datetime objects
    stock['date'] = pd.to_datetime(stock['date'])
    tweets['Date'] = pd.to_datetime(tweets['Date'])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filtering ticker and date
    stock_filtered = stock[(stock['symbol'] == ticker) & (stock['date'] >= start_date) & (stock['date'] <= end_date)]
    tweets_filtered = tweets[(tweets['Ticker'] == ticker) & (tweets['Date'] >= start_date) & (tweets['Date'] <= end_date)]

    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=("Price", "Sentiment"),
        vertical_spacing=0.3
    )

    # Line chart for stock prices
    fig.add_trace(
        go.Scatter(
            x=stock_filtered["date"],
            y=stock_filtered["close"],
            name="Close",
            mode="lines"
        ),
        row=1, col=1
    )

    # Candlestick chart for tweets sentiment
    fig.add_trace(go.Candlestick(
        x=tweets_filtered["Date"],
        open=tweets_filtered["Zero"],
        high=tweets_filtered["Positive Tweets"],
        low=tweets_filtered["Negative Tweets"],
        close=tweets_filtered["Zero"]),
        row=2, col=1
    )

    # Chart labels
    fig.update_layout(
        title=f"{ticker} Stock Price vs Tweet Sentiment",
        title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Price / Volume"
    )

    # Show chart
    return fig

def fetch_stock_data(ticker, start_date, end_date):
    # TODO -- Call an API to fetch daily stock prices for the given ticker,
    #         within the time frame defined by `start_date` and `end_date`
    return

### Run Sentiment Analysis

In [9]:
sent_apple = pd.read_csv('./data/AAPL.csv')
sent_apple.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL
2,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL
3,550443808606126081,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL
4,550443809700851716,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL


Apple

In [15]:
# sent_apple = pd.read_csv('./data/AAPL.csv')
# sent_apple['Sentiment'] = sent_apple['body'].apply(lambda text: calculate_sentiment_score(text))
sent_apple.to_csv('./data/sent-apple.csv', index=False)

Amazon

In [18]:
sent_amazon = pd.read_csv('./data/AMZN.csv')
sent_amazon['Sentiment'] = sent_amazon['body'].apply(lambda text: calculate_sentiment_score(text))
sent_amazon.to_csv('./data/sent-amazon.csv', index=False)

Google

In [3]:
sent_google = pd.read_csv('./data/GOOG.csv')
sent_google['Sentiment'] = sent_google['body'].apply(lambda text: calculate_sentiment_score(text))
sent_google.to_csv('./data/sent-google-goog.csv', index=False)

sent_google = pd.read_csv('./data/GOOGL.csv')
sent_google['Sentiment'] = sent_google['body'].apply(lambda text: calculate_sentiment_score(text))
sent_google.to_csv('./data/sent-google-googl.csv', index=False)

Microsoft

In [22]:
sent_microsoft = pd.read_csv('./data/MSFT.csv')
sent_microsoft['Sentiment'] = sent_microsoft['body'].apply(lambda text: calculate_sentiment_score(text))
sent_microsoft.to_csv('./data/sent-microsoft.csv', index=False)

In [13]:
test_apple = sent_apple.iloc[0:10000].copy()
test_apple['Sentiment'] = test_apple['body'].apply(lambda text: calculate_sentiment_score(text))
test_apple

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,Sentiment
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,0.0000
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,-0.8271
2,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,0.0000
3,550443808606126081,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,0.0000
4,550443809700851716,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,0.0000
...,...,...,...,...,...,...,...,...,...
9995,555005122871189505,Maximum_Pain_cm,1421158507,$AAPL high open interest range is 71.43 to 120...,0,0,0,AAPL,0.4588
9996,555005132296159233,garynielson,1421158509,The most popular cameras of 2014 according to ...,0,0,0,AAPL,0.4754
9997,555005539030011905,sportadore,1421158606,"Early movers: $SNDK, $HLT, $MET, $WWW, $AAPL &...",0,0,0,AAPL,0.0000
9998,555005616578908160,shawncj6,1421158625,$AAPL ripping premarket,0,0,0,AAPL,0.0000


### Fetch Stock Data

### Create and Run Gradio Interface

In [3]:
# Ticker list
tickers = ["AAPL", "AMZN", "GOOG", "GOOGL", "MSFT", "TSLA"]

# Custom css for the interface
custom_css = """
.center-text {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    height: 100vh;
    text-align: center;
}
"""

with gr.Blocks(theme="Monochrome", css=custom_css) as gui:
    # Home page
    with gr.Row(visible=True) as home_view:
        with gr.Column(elem_classes=["center-text"]):
            gr.Markdown("# Welcome to the Stock Price Chart Application")
            gr.Markdown("Use this model to generate stock price charts.")
            home_button = gr.Button("Go to Chart Generator")
    # Selection page
    with gr.Row(visible=False) as chart_view:
        with gr.Column(scale=1):
            company = gr.Dropdown(choices=tickers, label="Select Stock Ticker")
            instructions = gr.Markdown("## Select a date range between January 1, 2015, and December 31, 2019")
            start_date = Calendar(label="Select a Start Date", type="datetime", value="2015-01-01")
            end_date = Calendar(label="Select an End Date", type="datetime",value="2019-12-31")
            generate_button = gr.Button("Generate Chart", scale=1)
            back_button = gr.Button("Back to Home", scale=1)
    # Plot page
    with gr.Row(visible=False) as plot_view:
        with gr.Column(scale=75):
            plot_output = gr.Plot()
        with gr.Column(scale=1):
            back_to_chart_button = gr.Button("Back to Chart Generator")
    

    # Button functions
    home_button.click(lambda: (gr.update(visible=False), gr.update(visible=True)), None, [home_view, chart_view])
    back_button.click(lambda: (gr.update(visible=True), gr.update(visible=False)), None, [home_view, chart_view])
    generate_button.click(lambda: (gr.update(visible=False), gr.update(visible=True)), None, [chart_view, plot_view])
    generate_button.click(lambda ticker, start, end: generate_chart(ticker, start, end) if ticker else None, [company, start_date, end_date], plot_output)
    back_to_chart_button.click(lambda: (gr.update(visible=True), gr.update(visible=False)), None, [chart_view, plot_view])

gui.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://101215aadb4893b2c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


c:\Users\bapej\anaconda3\envs\dev\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

